In [1]:
from PIL import Image
import numpy as np
import json

from pattern_creation import get_pattern_style, get_pattern_img_by_style
from apply_pattern import apply_pattern
from random import shuffle
from tqdm import tqdm
from pathlib import Path
import pandas as pd

In [2]:
DEBUG = False

In [3]:
def log( message ):
    if DEBUG:
        print( message )

In [4]:
def get_img_path( dir_path ):
    i = 1
    file_path = dir_path / f'out_{ str(i).zfill(3) }.jpg'

    while file_path.exists():
        i += 1
        file_path = dir_path / f'out_{ str(i).zfill(3) }.jpg'

    return file_path

In [5]:
with open('./config_deform.json') as config_file:
    config = json.load(config_file)

In [6]:
rows = []

for i in tqdm(range(10)):
    row = {
        'image' : "gray_tile",
        'img_path' : "",
        'method' : 'soft_light', #'screen' if random.random() > 0.5 else 'multiply',
        'ssim' : -1,
        'bbox' : [0,0,1000,1000],
        'overlay_intensity_C' : config['overlay_intensity'][0],
        'overlay_intensity_M' : config['overlay_intensity'][1],
        'overlay_intensity_Y' : config['overlay_intensity'][2],
        'overlay_intensity_K' : config['overlay_intensity'][3]
    }
    
    pattern_style = get_pattern_style(config)
    for key in pattern_style:
        row[key] = pattern_style[key]

    rows.append(row)
    pattern_img = get_pattern_img_by_style(row, config)
    
    softlight_test,pattern_mask, ssim_value_K = apply_pattern(
        (np.ones((5000,5000)) * 100).astype('uint8'),
        pattern_img,
        (2500,2500),
        config,
        method='soft_light',
        overlay_weight=0.5,
        log=log 
    )
    
    Image.fromarray(softlight_test).save( get_img_path(Path('.')) , dpi=(600,600))

  0%|                                                    | 0/10 [10:24<?, ?it/s]


OSError: [Errno 89] Operation canceled

In [ ]:
pd.DataFrame.from_dict(rows)